In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy
from scipy.spatial.distance import pdist, squareform
import sklearn
import matplotlib.pyplot as plt
from collections import Counter
import tqdm
import requests
import time

In [2]:
def load_network_sparse(net_file,ngene):
    ppi_df = pd.read_csv(net_file,header=None,sep='\t')
    A = np.zeros((ngene,ngene))
    row_idx = ppi_df.iloc[:,0].values -1 
    col_idx = ppi_df.iloc[:,1].values -1
    A[row_idx, col_idx] = ppi_df.iloc[:,2].values
    assert (A == A.T).all()
    zero_rows = np.all(A == 0, axis=1)
    diag_indices = np.arange(ngene)
    A[diag_indices[zero_rows], diag_indices[zero_rows]] = 1
    return A

In [3]:
def load_all_nets(ppi_files,n_gene):
    '''
    parameters:
    - ppi_files: [str, str, ...], list of network file paths, each file should contain three columns: [protein1, protein2, score]
    - ref_gene_file: str, file path, the file contains all genes, one gene per line
    output:
    - nets: n_file x n_gene x n_gene array with ppi networks
    '''
    n_file = len(ppi_files)
    nets = np.zeros((n_file,n_gene,n_gene))
    for i in range(n_file):
        A = load_network_sparse(ppi_files[i],n_gene)
        nets[i,:,:] = A
    return nets

In [4]:
def compute_rwr_original_sparse(ppi_files,restart_prob,ngene,nets):
    ''' 
    - ppi_files: list of network file paths
    - restart_prob: RWR restart probability
    - ngene: number of genes
    output:
    - walks: for the i-th RWR result, [i,:,:], each column is the stationary distribution of a node
    '''
    n_file = len(ppi_files)
    e = np.ones(ngene)
    I = np.eye(ngene)
    walks = np.zeros((n_file,ngene,ngene))
    for i in range(n_file):
        A = nets[i,:,:]
        d = A @ e
        P = A / d # transition matrix
        W = (I - (1 - restart_prob) * P)
        W = np.linalg.inv(W)
        W = W * restart_prob 
        walks[i,:,:] = W
    return walks

In [5]:
def svd_embed_sparse_func(walks, ngene, embed_dim):
    n_net = walks.shape[0]
    mat = np.zeros((ngene,ngene))
    W_updated = np.zeros_like(walks)
    for i in range(n_net):
        W = walks[i,:,:]
        W[W<=1e-8] = 0
        W = np.log(W, where = W > 1e-8)
        W_updated[i,:,:] = W
        tmp = W.T @ W
        mat = mat + tmp
    eigenvalues, eigenvectors = scipy.sparse.linalg.eigs(mat,k=embed_dim)
    x = np.diag(np.sqrt(np.sqrt(eigenvalues))) @ eigenvectors.T
    return np.real(x)

In [6]:
def load_train_test_anno(rand,fold,org,ont_type,ont_size1,ont_size2):
    '''
    predifined fold splits
    - rand: 1 2 3 4 5
    - fold: 1 2 3 4 5
    - org: "Ecoli" or "yeast"
    '''
    file_name = 'data/train_test_split/'+org+'/rand' + str(rand) +'/fold' + str(fold) + '_' + ont_type+ '_' +  str(ont_size1)+ '_' +  str(ont_size2)+ '_train_anno.txt'
    train = pd.read_csv(file_name,header=None,sep = '\t')
    file_name = 'data/train_test_split/'+org+'/rand' + str(rand) +'/fold' + str(fold) + '_' + ont_type+ '_' +  str(ont_size1)+ '_' +  str(ont_size2)+ '_test_anno.txt'
    test = pd.read_csv(file_name,header=None,sep = '\t')
    return train.to_numpy(), test.to_numpy()

In [7]:
def augment_graph(nets, ngene, gene_clusters, mustlink_weight, cannotlink_weight):
    '''
    - nets: original adjacency matrices directly read from PPI files
    - gene_clusters: (num_clusers, num_genes), binary matrix indicating which gene belongs to which clusters
    '''
    n_nets = nets.shape[0]
    n_clusters = gene_clusters.shape[0]
    augmented = np.zeros((n_nets,(ngene+n_clusters),(ngene+n_clusters)))
    for i in range(n_nets):
        A = nets[i,:,:]
        A_block = np.block([[A,mustlink_weight*gene_clusters.T],[mustlink_weight*gene_clusters,cannotlink_weight*np.ones((n_clusters,n_clusters))]])
        np.fill_diagonal(A_block,0)
        zero_rows = np.all(np.absolute(A_block) == 0, axis=1)
        diag_indices = np.arange(ngene+n_clusters)
        A_block[diag_indices[zero_rows], diag_indices[zero_rows]] = 1
        augmented[i,:,:] = A_block
    return augmented

In [8]:
def augmented_RWR(augmented_nets, restart_prob):
    '''
    RWR for augmented graph which contains negative edge weights
    '''
    n_nets = augmented_nets.shape[0]
    n_nodes = augmented_nets.shape[1]
    augmented_walks = np.zeros((n_nets,n_nodes,n_nodes))
    e = np.ones(n_nodes)
    for i in range(n_nets):
        A = augmented_nets[i,:,:]
        d = np.absolute(A) @ e
        L = np.diag(d) - (1-restart_prob)*A
        L_inv = np.linalg.inv(L)
        W = restart_prob*(np.diag(d) @ L_inv)
        augmented_walks[i,:,:] = W
    return augmented_walks


In [9]:
def augmented_SVD_with_cannolink(aug_walks, embed_dim):
    n_net = aug_walks.shape[0]
    n_node = aug_walks.shape[1]
    mat = np.zeros((n_node,n_node))
    W_updated = np.zeros_like(aug_walks)
    for i in range(n_net):
        W = aug_walks[i,:,:]
        min_entry = W.min()
        if min_entry > 0:
            min_entry = 0.0
        W = W - min_entry
        W[W<=1e-8] = 0
        W = np.log(W, where = W > 1e-8)
        W_updated[i,:,:] = W
        tmp = W.T @ W
        mat = mat + tmp
    eigenvalues, eigenvectors = scipy.sparse.linalg.eigs(mat,k=embed_dim)
    x = np.diag(np.sqrt(np.sqrt(eigenvalues))) @ eigenvectors.T
    return np.real(x)

In [10]:
def get_knn_ind(embed,train_anno):
    '''
    parameters:
    - embed: (dim, num_gene), protein embeddings
    - train_anno: annotations for training proteins
    output:
    dist_mat: n_gene x n_gene
    sorted_ind: ngene x (ngene-1), top n labels
    '''
    n_gene = train_anno.shape[1]
    dist_mat = squareform(pdist(embed.T))
    dist_mat = dist_mat[:n_gene,:n_gene] # symmetrical
    np.fill_diagonal(dist_mat, 1e8)
    sorted_ind = np.argsort(dist_mat, axis=1)
    
    return dist_mat, sorted_ind


In [11]:
def majority_vote(dist_mat, knn_mat, train_anno, test_anno,k, weighted=True):
    '''
    parameters:
    - dist_mat: n_gene x n_gene
    - knn_mat: ngene x (ngene-1), sorted labels
    - train_anno: n_label x n_gene
    - test_anno: n_label x n_gene
    - k: number of nearest neighbors
    - weighted: boolean, whether doing weighted majority vote or not
    output:
    - final_scores: n_label x n_test, normalized scores of each label
    - num_voters: vector of numbers of voting nodes
    '''
    train_idx = np.where(sum(train_anno)>0)[0]
    test_idx = np.where(sum(test_anno)>0)[0]
    final_scores = np.zeros((train_anno.shape[0],len(test_idx)))
    num_voters = []
    updated_voters = []
    c = 0
    for index, i in enumerate(test_idx):
        nn = knn_mat[i,:k]
        nn_labeled = nn[np.isin(nn, train_idx)] 
        
        if len(nn_labeled) == 0: # if within the first k neighbors, no neighbor is labeled, then use the nearest neighbor with label
            voting_node = knn_mat[i,:][np.isin(knn_mat[i,:], train_idx)][0]
            scores = np.array(train_anno[:,voting_node])
            scores = scores / sum(scores)
            num_voters.append(len(nn_labeled))
            tmp = [voting_node]
            updated_voters.append(tmp)
        else:
            votes = np.array(train_anno[:,nn_labeled])
            if weighted:
                d = dist_mat[i,nn_labeled]
                d = d[np.nonzero(d)]
                votes = np.array(train_anno[:,nn_labeled[np.nonzero(d)]])
                tmp = nn_labeled[np.nonzero(d)]
                updated_voters.append(tmp)
                num_voters.append(len(d))
                if len(d) == 0:
                    c += 1
                    voting_node = np.random.choice(train_idx)
                    scores = np.array(train_anno[:,voting_node])
                    scores = scores / sum(scores)
                else:
                    weights = 1 / d
                    scores = votes @ weights.T
                    scores = scores / sum(scores)
            else:
                num_voters.append(len(nn_labeled))
                updated_voters.append(nn_labeled)
                scores = np.sum(votes,axis=1)
                scores = scores / sum(scores)
        
        final_scores[:,index] = np.squeeze(scores)
        # print(c)
    return final_scores, num_voters,updated_voters


In [12]:
def acc_top1_pred(test_scores, test_anno):
    '''
    for each test gene, find the label with the highest predicted score, use it as the predicted label
    accuracy is defined as (#predicted label in test true labels) / (#test genes)
    problems: if there's a tie, the one with smaller index will be used
    parameters:
    - test_scores: n_label x n_test
    - test_anno: n_label x n_gene
    output:
    - acc: accuracy score
    '''
    test_idx = np.where(sum(test_anno)>0)[0]
    zero_idx = np.where(np.sum(test_scores,axis=0)==0)[0]
    mask = np.ones(len(test_idx), dtype=bool)
    mask[zero_idx] = False
    test_anno = test_anno[:,test_idx] # n_label x n_test
    sorted_index = np.argsort(-1*test_scores,axis=0) # n_label x n_test, with row 0 the highest predicted label for each gene
    true_pred = test_anno[sorted_index[0,:], np.arange(test_anno.shape[1])]
    true_pred = true_pred[mask]
    acc = np.mean(true_pred)
    return acc,true_pred
    

In [13]:
def f1_auprc_pred(test_scores, test_anno,top_n):
    '''
    for each test gene, find the labels with the top_n highest predicted scores, use them as the predicted labels
    f1 is defined as 2*TP / 2*TP + FP + FN
    probelms: if there's a tie, the one with smaller index will be used, only top n predictions will be considered, it will increase the number of FN
    parameters:
    - test_scores: n_label x n_test
    - test_anno: n_label x n_gene
    - top_n: int, the number of labels to be predicted
    output:
    - acc: accuracy score
    '''
    test_idx = np.where(sum(test_anno)>0)[0]
    zero_idx = np.where(np.sum(test_scores,axis=0)==0)[0]

    mask = np.ones(len(test_idx), dtype=bool)
    mask[zero_idx] = False
    test_anno = test_anno[:,test_idx] # n_label x n_test
    
    test_anno = test_anno[:,mask]
    test_scores = test_scores[:,mask]
    sorted_index = np.argsort(-1*test_scores,axis=0) # n_label x n_test, with row 0 the highest predicted label for each gene
    top_ind = sorted_index[:top_n,:].flatten()
    pred = np.zeros_like(test_anno)
    cols = np.tile(np.arange(test_anno.shape[1]), top_n)
    pred[top_ind, cols] = 1
    f1 = sklearn.metrics.f1_score(test_anno.flatten(),pred.flatten())
    precision, recall, thresholds = sklearn.metrics.precision_recall_curve(test_anno.flatten(), pred.flatten())
    auprc = sklearn.metrics.auc(recall, precision)
    return f1, auprc
    


In [14]:
'''
for each augmented node, randomly choose a fixed number of nodes to connect to
the number of genes that each augmented node connects to are the same except for the last one
'''
def random_split_vector(train_anno,n_gene, num_sub_vectors,seed=None):
    if seed is not None:
        np.random.seed(seed)
        
    input_vector = np.where(sum(train_anno)>0)[0]
    
    if num_sub_vectors <= 0 or num_sub_vectors > len(input_vector):
        raise ValueError("Invalid number of sub-vectors")
    
    shuffled_vector = np.random.permutation(input_vector)
    sub_vector_size = len(shuffled_vector) // num_sub_vectors
    
    group_matrix = np.zeros((num_sub_vectors, len(input_vector)), dtype=int)
    res_matrix = np.zeros((num_sub_vectors, n_gene), dtype=int)
    
    start_index = 0
    for i in range(num_sub_vectors):
        end_index = start_index + sub_vector_size
        
        if i == num_sub_vectors - 1:
            end_index = len(shuffled_vector)
        
        selected_indices = shuffled_vector[start_index:end_index] 
        group_matrix[i, np.isin(shuffled_vector, selected_indices)] = 1
        
        start_index = end_index
    
    res_matrix[:,shuffled_vector] = group_matrix
    
    return res_matrix

In [15]:
def run_pipeline(ppi_files,n_gene,method=None,restart_prob=None,embed_dim=None,rand=None,org=None,n_fold=None,k=None,ont_type=None,ont_size1=None,ont_size2=None,n_cluster=None):
    ''' 
    parameters:
    - ppi_files: list of str, list of file paths to ppi networks
    - n_gene: int, number of genes
    - method: list of str, one or more of Mashup, REPEL
    - restart_prob: float, RWR restart probability
    - embed_dim: int, number of dimension
    - rand: int, random split
    - org: str, "yeast" or "Ecoli" 
    - n_fold: int, total number of folds
    - k: int, number of nearest neighbors to be considered
    - ont_type: str, bp or mf or cc
    - ont_size1: int, 11, 31, 101
    - ont_size2: int, 30, 100, 300
    - n_cluster: int, number of random augmented nodes
    output:
    - performance_dict: a dictionary contains list of performances for all methods
    '''

    performance_dict = {}

    for m in method:
        m_acc = m + "_acc"
        m_f1 = m + "_f1"
        m_auprc = m + "_auprc"
        performance_dict[m_acc] = []
        performance_dict[m_f1] = []
        performance_dict[m_auprc] = []
        for i in range(n_fold):
            print("fold: ", i+1)
            train_anno, test_anno = load_train_test_anno(rand,i+1,org,ont_type,ont_size1,ont_size2)
            if m == "Mashup":
                print("Mashup")
                nets = load_all_nets(ppi_files,n_gene)
                walks = compute_rwr_original_sparse(ppi_files,restart_prob,n_gene,nets)
                x = svd_embed_sparse_func(walks, n_gene, embed_dim)
                dist_mat, knn = get_knn_ind(x,train_anno)
                scores, _, _ = majority_vote(dist_mat, knn, train_anno, test_anno,k)
                acc,_ = acc_top1_pred(scores, test_anno)
                f1, auprc = f1_auprc_pred(scores, test_anno,3)
                performance_dict[m_acc].append(acc)
                performance_dict[m_f1].append(f1)
                performance_dict[m_auprc].append(auprc)
            elif m == "REPEL":
                print("REPEL")
                nets = load_all_nets(ppi_files,n_gene)
                rand_cluster = random_split_vector(train_anno,n_gene, n_cluster,seed=None)
                rand_graph = augment_graph(nets, n_gene, rand_cluster, 1, -1)
                rand_rwr_res = augmented_RWR(rand_graph, restart_prob)
                mat_rand_x = augmented_SVD_with_cannolink(rand_rwr_res, embed_dim)
                dist_mat, knn = get_knn_ind(mat_rand_x,train_anno)
                scores, _,_ = majority_vote(dist_mat, knn, train_anno, test_anno,k, weighted=True)
                acc,_ = acc_top1_pred(scores, test_anno)
                f1, auprc = f1_auprc_pred(scores, test_anno,3)
                performance_dict[m_acc].append(acc)
                performance_dict[m_f1].append(f1)
                performance_dict[m_auprc].append(auprc)

            else:
                print("Haven't implemented yet")
                return
    return performance_dict


In [16]:
def write_log(performance_dict,rand,org,ont_type,ont_size1,ont_size2,save_path=None):
    with open(save_path,"a") as f:
        tmp = org + " " + "rand " + str(rand) + " " + ont_type + " " + str(ont_size1) + " " + str(ont_size2) 
        f.write(tmp)
        f.write("\n")
        for k, v in performance_dict.items():
            f.write(k)
            f.write(" ")
            for i in v:
                f.write(f"{i:.4f}")
                f.write(" ")
            f.write("\n")

For yeast

In [17]:
# yeast
string_nets = ['neighborhood', 'fusion', 'cooccurence', 'coexpression', 'experimental', 'database']
ppi_files = []
for net in string_nets:
    tmp = 'data/networks/yeast/yeast_string_'+net+'_adjacency.txt'
    ppi_files.append(tmp)

all_genes = pd.read_csv("data/annotations/yeast/go_yeast_ref_genes.txt",header=None)
all_genes = list(all_genes.iloc[:,0].values)
n_gene = len(all_genes)
print(ppi_files)
print(n_gene)

['data/networks/yeast/yeast_string_neighborhood_adjacency.txt', 'data/networks/yeast/yeast_string_fusion_adjacency.txt', 'data/networks/yeast/yeast_string_cooccurence_adjacency.txt', 'data/networks/yeast/yeast_string_coexpression_adjacency.txt', 'data/networks/yeast/yeast_string_experimental_adjacency.txt', 'data/networks/yeast/yeast_string_database_adjacency.txt']
6400


In [ ]:
method=['REPEL','Mashup']
restart_prob=0.5
embed_dim=400
org="yeast"
n_fold=5
k=10
ont_type_list=["bp","mf","cc"]
ont_size1_list=[11,31,101]
ont_size2_list=[30,100,300]
n_cluster=15
for rand in range(1,6):
    print("rand: ",rand)
    for ont_type in ont_type_list:
        print("ont_type")
        for c in range(3):
            ont_size1 = ont_size1_list[c]
            ont_size2 = ont_size2_list[c]
            performance_dict = run_pipeline(ppi_files,n_gene,method=method,restart_prob=restart_prob,embed_dim=embed_dim,rand=rand,org=org,n_fold=n_fold,k=k,ont_type=ont_type,ont_size1=ont_size1,ont_size2=ont_size2,n_cluster=n_cluster)
            write_log(performance_dict,rand,org,ont_type,ont_size1,ont_size2,save_path="yeast_result_log.txt")
            print(performance_dict)
    #         break
    #     break
    # break


rand:  1
ont_type
fold:  1
REPEL
fold:  2
REPEL
fold:  3
REPEL
fold:  4
REPEL
fold:  5
REPEL
fold:  1
Mashup
fold:  2
Mashup
fold:  3
Mashup
fold:  4
Mashup
fold:  5
Mashup
{'REPEL_acc': [np.float64(0.5), np.float64(0.48773006134969327), np.float64(0.463452566096423), np.float64(0.46248085758039814), np.float64(0.5116640746500778)], 'REPEL_f1': [0.30425531914893617, 0.30364900086880975, 0.29443838604143946, 0.2880874316939891, 0.3233033164946189], 'REPEL_auprc': [np.float64(0.31380270737186944), np.float64(0.3127148273903542), np.float64(0.3041420884436786), np.float64(0.296206414448308), np.float64(0.3330274441988971)], 'Mashup_acc': [np.float64(0.3685800604229607), np.float64(0.3450920245398773), np.float64(0.3343701399688958), np.float64(0.3430321592649311), np.float64(0.3343701399688958)], 'Mashup_f1': [0.23361702127659575, 0.22849695916594265, 0.22813522355507088, 0.22688524590163933, 0.2249066549527784], 'Mashup_auprc': [np.float64(0.2415997431897603), np.float64(0.23600749832405

For E. coli

In [19]:
# for E.coli
string_nets = ['neighborhood', 'fusion', 'cooccurence', 'coexpression', 'experimental', 'database']
ppi_files = []
for net in string_nets:
    tmp = 'data/networks/Ecoli/ecoli_string_'+net+'_adjacency.txt'
    ppi_files.append(tmp)

all_genes = pd.read_csv("data/annotations/Ecoli/ecoli_ref_genes.txt",header=None,sep='\t')
all_genes = list(all_genes.iloc[:,1].values)
n_gene = len(all_genes)
print(ppi_files)
print(n_gene)


['data/networks/Ecoli/ecoli_string_neighborhood_adjacency.txt', 'data/networks/Ecoli/ecoli_string_fusion_adjacency.txt', 'data/networks/Ecoli/ecoli_string_cooccurence_adjacency.txt', 'data/networks/Ecoli/ecoli_string_coexpression_adjacency.txt', 'data/networks/Ecoli/ecoli_string_experimental_adjacency.txt', 'data/networks/Ecoli/ecoli_string_database_adjacency.txt']
4140


In [20]:
method=['Mashup','REPEL']
restart_prob=0.5
embed_dim=400
org="Ecoli"
n_fold=5
k=10
ont_type_list=["bp","mf","cc"]
ont_size1_list=[11,31,101]
ont_size2_list=[30,100,300]
n_cluster=15
for rand in range(1,6):
    print("rand: ",rand)
    for ont_type in ont_type_list:
        print("ont_type")
        for c in range(3):
            ont_size1 = ont_size1_list[c]
            ont_size2 = ont_size2_list[c]
            performance_dict = run_pipeline(ppi_files,n_gene,method=method,restart_prob=restart_prob,embed_dim=embed_dim,rand=rand,org=org,n_fold=n_fold,k=k,ont_type=ont_type,ont_size1=ont_size1,ont_size2=ont_size2,n_cluster=n_cluster)
            write_log(performance_dict,rand,org,ont_type,ont_size1,ont_size2,save_path="Ecoli_result_log.txt")
            print(performance_dict)
            break
        break
    break



rand:  1
ont_type
fold:  1
Mashup
fold:  2
Mashup
fold:  3
Mashup
fold:  4
Mashup
fold:  5
Mashup
fold:  1
REPEL
fold:  2
REPEL
fold:  3
REPEL
fold:  4
REPEL
fold:  5
REPEL
{'Mashup_acc': [np.float64(0.4166666666666667), np.float64(0.47474747474747475), np.float64(0.45569620253164556), np.float64(0.5012658227848101), np.float64(0.4531645569620253)], 'Mashup_f1': [0.3101030927835052, 0.3415821501014199, 0.3191407748369774, 0.37645174209050863, 0.33726127590410404], 'Mashup_auprc': [np.float64(0.31310573851331064), np.float64(0.34488730032756837), np.float64(0.32516911124506065), np.float64(0.38024772694230863), np.float64(0.340606303100847)], 'REPEL_acc': [np.float64(0.5984848484848485), np.float64(0.6136363636363636), np.float64(0.6278481012658228), np.float64(0.6658227848101266), np.float64(0.620253164556962)], 'REPEL_f1': [0.43381443298969075, 0.4486815415821501, 0.4158036056770234, 0.47577092511013214, 0.44697277529459567], 'REPEL_auprc': [np.float64(0.4363665672780467), np.float64(